# Hierarchical model support

**Author: Lukas Breitwieser**<br>
Some models require to update certain agents before others.
In this tutorial we show how to execute operations first for large agents and afterwards for small ones.
Lastly, we demonstrate how to run a different set of operations for large and for small agents.

Let's start by setting up BioDynaMo notebooks.

In [ ]:
%jsroot on
gROOT->LoadMacro("${BDMSYS}/etc/rootlogon.C");

To make this demo easier to understand, we turn off multi-threading and load balancing. 

In [ ]:
#include <omp.h>
omp_set_num_threads(1);
ThreadInfo::GetInstance()->Renew();
auto* scheduler = simulation.GetScheduler();
scheduler->UnscheduleOp(scheduler->GetOps("load balancing")[0]);

We create a new agent operation which prints out its name and the diameter of the agent it is processing

In [ ]:
struct TestOp : public AgentOperationImpl {
  BDM_OP_HEADER(TestOp);
  void operator()(Agent* agent) override {
    std::cout << name << " processing agent with diameter " 
              << agent->GetDiameter() << endl; 
  }
  std::string name = "";
};
OperationRegistry::GetInstance()->AddOperationImpl(
    "test_op", OpComputeTarget::kCpu, new TestOp());

We create four agents with diameter `{20, 10, 20, 10}`

In [ ]:
auto* ctxt = simulation.GetExecutionContext();
for (int i = 0; i < 4; ++i) {
    double diameter = i % 2 == 0 ? 20 : 10;
    ctxt->AddAgent(new SphericalAgent(diameter));
}

We add the new operation to the simulation

In [ ]:
auto* op1 = NewOperation("test_op");
scheduler->ScheduleOp(op1);

Let's simulate one time step and observe the default behavior of BioDynaMo.
We expect that the agents are processed in the order they were added (`{20, 10, 20, 10}`)

In [ ]:
scheduler->Simulate(1);

Now we want to define the group of large and small agents and tell BioDynaMo that large agents should be processed before small ones. 

This can be done with the following three lines of code.

In [ ]:
auto small_filter = L2F([](Agent* a) { return a->GetDiameter() < 15; });
auto large_filter = L2F([](Agent* a) { return a->GetDiameter() >= 15; });
scheduler->SetAgentFilters({&large_filter, &small_filter});

Let's observe if the output has changed. We expect to see first the large agents `{20, 20}`,  followed by the small ones `{10, 10}`.

In [ ]:
scheduler->Simulate(1);

Let's create two more instances of our `TestOp`.
We define that:
* `op1` should be run for all agents (large and small).
* `op2` only for small agents
* `op3` only for large agents

In [ ]:
auto* op2 = NewOperation("test_op");
auto* op3 = NewOperation("test_op");

op1->GetImplementation<TestOp>()->name = "OpAll      ";
op2->GetImplementation<TestOp>()->name = "OpOnlySmall";
op3->GetImplementation<TestOp>()->name = "OpOnlyLarge";

op2->SetExcludeFilters({&large_filter});
op3->SetExcludeFilters({&small_filter});

scheduler->ScheduleOp(op2);
scheduler->ScheduleOp(op3);

Now we want to execute another time step with the updated model. We expect that for each agent two operations will be executed. 

For large agents `OpAll` and `OpOnlyLarge` and for small agents `OpAll` and `OpOnlySmall`.
As before, we expect that first all large agents are executed, followed by all small agents.

In [ ]:
scheduler->Simulate(1);